### pandas를 이용한 데이터 분석 맛보기

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/lending-club-loan-data/loan.csv", sep=",")


/Users/jang-eun-a/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(887379, 74)

In [5]:
df.columns[:10]

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv',
       u'term', u'int_rate', u'installment', u'grade', u'sub_grade'],
      dtype='object')

In [6]:
df2 = df[["loan_amnt","loan_status","grade","int_rate","term"]]

In [7]:
# df2의 맨앞 5개 행 조회
df2.head()

,loan_amnt,loan_status,grade,int_rate,term
0,5000.0,Fully Paid,B,10.65,36 months
1,2500.0,Charged Off,C,15.27,60 months
2,2400.0,Fully Paid,C,15.96,36 months
3,10000.0,Fully Paid,C,13.49,36 months
4,3000.0,Current,B,12.69,60 months


In [8]:
# 맨뒤 5개 행 조회
df2.tail()

,loan_amnt,loan_status,grade,int_rate,term
887374,10000.0,Current,B,11.99,36 months
887375,24000.0,Current,B,11.99,36 months
887376,13000.0,Current,D,15.99,60 months
887377,12000.0,Current,E,19.99,60 months
887378,20000.0,Current,B,11.99,36 months


In [9]:
df2["loan_status"].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [10]:
df2["grade"].unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [11]:
df2["term"].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [12]:
df2.shape

(887379, 5)

In [13]:
# 결측값 제거하기
df2 = df2.dropna(how="any")

In [14]:
df2.shape

(887379, 5)

In [15]:
# 36개월 대출과 60개월 대툴의 대출 총액 파악하기

# 기간에 따른 대출 총액을 계산할 딕션 생성
term_to_loan_amnt_dict = {}
# 대출 상품의 유니크한 기간 추출
uniq_terms = df2["term"].unique()

In [16]:
for term in uniq_terms:
    loan_amnt_sum = df2.loc[df2["term"] == term, "loan_amnt"].sum()
    term_to_loan_amnt_dict[term] = loan_amnt_sum

In [17]:
term_to_loan_amnt_dict

{' 36 months': 7752507375.0, ' 60 months': 5341004575.0}

In [18]:
# 딕션을 시리즈로 변환
term_to_loan_amnt = pd.Series(term_to_loan_amnt_dict)

In [19]:
term_to_loan_amnt

 36 months    7.752507e+09
 60 months    5.341005e+09
dtype: float64

In [20]:
# 각 대출 상태(불량/우량)에 따른 대출 등급 분포 파악
df2.head()

,loan_amnt,loan_status,grade,int_rate,term
0,5000.0,Fully Paid,B,10.65,36 months
1,2500.0,Charged Off,C,15.27,60 months
2,2400.0,Fully Paid,C,15.96,36 months
3,10000.0,Fully Paid,C,13.49,36 months
4,3000.0,Current,B,12.69,60 months


In [21]:
df2["loan_status"].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [22]:
# 불량한 상태의 것 분류
total_status_category = df2["loan_status"].unique()
bad_status_category = total_status_category[[1,3,4,5,6,8]]

In [23]:
bad_status_category

array(['Charged Off', 'Default', 'Late (31-120 days)', 'In Grace Period',
       'Late (16-30 days)',
       'Does not meet the credit policy. Status:Charged Off'], dtype=object)

In [24]:
df2["bad_loan_status"] = df2["loan_status"].isin(bad_status_category)

In [25]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_status
0,5000.0,Fully Paid,B,10.65,36 months,False
1,2500.0,Charged Off,C,15.27,60 months,True
2,2400.0,Fully Paid,C,15.96,36 months,False
3,10000.0,Fully Paid,C,13.49,36 months,False
4,3000.0,Current,B,12.69,60 months,False


In [26]:
bad_loan_status_to_grades = df2.loc[df2["bad_loan_status"] == True, "grade"].value_counts()

In [27]:
bad_loan_status_to_grades

C    19054
D    15859
B    13456
E     9745
F     4383
A     3663
G     1269
Name: grade, dtype: int64

In [28]:
# 인덱스 정렬
bad_loan_status_to_grades.sort_index()

A     3663
B    13456
C    19054
D    15859
E     9745
F     4383
G     1269
Name: grade, dtype: int64

In [29]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_status
0,5000.0,Fully Paid,B,10.65,36 months,False
1,2500.0,Charged Off,C,15.27,60 months,True
2,2400.0,Fully Paid,C,15.96,36 months,False
3,10000.0,Fully Paid,C,13.49,36 months,False
4,3000.0,Current,B,12.69,60 months,False


In [30]:
# 대출 총액과 대출 이자율 간의 상관관계 파악
df2["loan_amnt"].corr(df2["int_rate"])

0.145023099298839

In [31]:
bad_loan_status_to_grades.to_csv("bad_loans_status.csv", sep=",")